In [1]:
import pandas as pd
import os

# employment outlook 

In [2]:
# Load the cleaned data from the specified sheet
file_path = 'data/occupation-profiles-data.xlsx'
data = pd.read_excel(file_path, sheet_name='employment_data_clean')


In [3]:
data.head()

,ANZSCO \ncode,Occupation,Employment_2011,Employment_2012,Employment_2013,Employment_2014,Employment_2015,Employment_2016,Employment_2017,Employment_2018,...,ACT (%),Age 15 - 19,Age 20 - 24,Age 25 - 34,Age 35 - 44,Age 45 - 54,Age 55 - 59,Age 60 - 64,Age 65 +,Median Age
0,1111,Chief Executives and Managing Directors,66200.0,69100.0,61600.0,46900.0,64000.0,51700.0,47400.0,63000.0,...,2.0,NaN,0.5,7.9,22.7,34.0,14.6,10.2,10.1,50
1,1112,General Managers,50300.0,49800.0,58100.0,53400.0,52800.0,46800.0,38900.0,46800.0,...,2.7,0.1,0.9,12.3,29.6,33.0,11.5,6.9,5.7,46
2,1113,Legislators,2100.0,1500.0,2500.0,1200.0,4700.0,1900.0,2700.0,3800.0,...,1.9,NaN,1.2,8.0,16.7,27.9,13.2,14.2,18.8,53
3,1211,Aquaculture Farmers,1600.0,1700.0,2000.0,1900.0,2300.0,4000.0,3900.0,1100.0,...,NaN,2.5,7.0,17.8,21.0,22.3,10.7,8.8,10.0,45
4,1212,Crop Farmers,45100.0,37200.0,47400.0,40900.0,42600.0,37400.0,43500.0,40100.0,...,0.1,0.8,3.0,11.8,17.8,23.9,12.7,11.3,18.7,52


In [4]:
data = data.rename(columns={'ANZSCO \ncode': 'ANZSCO code','Age 65 +  ':'Age 65 +'})

In [5]:
data.columns

Index(['ANZSCO code', 'Occupation', 'Employment_2011', 'Employment_2012',
       'Employment_2013', 'Employment_2014', 'Employment_2015',
       'Employment_2016', 'Employment_2017', 'Employment_2018',
       'Employment_2019', 'Employment_2020', 'Employment_2021',
       'Projected Employment level', 'Projected Employment Growth',
       'Projected Employment Growth (%)', 'Future Growth Rating', 'Male Share',
       'Female Share', 'Full-time Share', 'Part-time Share', 'industry',
       'NSW (%)', 'VIC (%)', 'QLD (%)', 'SA (%)', 'WA (%)', 'TAS (%)',
       'NT (%)', 'ACT (%)', 'Age 15 - 19', 'Age 20 - 24', 'Age 25 - 34',
       'Age 35 - 44', 'Age 45 - 54', 'Age 55 - 59', 'Age 60 - 64', 'Age 65 +',
       'Median Age'],
      dtype='object')

In [6]:
columns_to_keep = ['ANZSCO code', 'Occupation']
df_melted = pd.melt(data, 
                    id_vars=columns_to_keep, 
                    var_name="Year/Metric", 
                    value_name="Value")



In [7]:
# Display the first few rows to verify the transformation
df_melted.tail()


,ANZSCO code,Occupation,Year/Metric,Value
13093,8994,Motor Vehicle Parts and Accessories Fitters,Median Age,34
13094,8995,Printing Assistants and Table Workers,Median Age,46
13095,8996,Recycling and Rubbish Collectors,Median Age,46
13096,8997,Vending Machine Attendants,Median Age,37
13097,8999,Other Miscellaneous Labourers,Median Age,43


In [8]:
def split_year_metric(row):
    if 'Employment_' in row['Year/Metric']:
        metric, year = row['Year/Metric'].split('_')
        return pd.Series([year, metric])
    else:
        return pd.Series([None, row['Year/Metric']])

df_melted[['Year', 'Metric']] = df_melted.apply(split_year_metric, axis=1)


In [9]:
df_cleaned = df_melted[['ANZSCO code', 'Occupation', 'Year', 'Metric', 'Value']]


In [10]:
# value fill

# Fill missing values in 'Year' column where 'Metric' is 'Projected Employment level' with '2026'
df_cleaned.loc[(df_cleaned['Metric'] == 'Projected Employment level') & (df_cleaned['Year'].isnull()), 'Year'] = '2026'

In [11]:

# Display all rows where 'Value' is NaN
nan_rows = df_cleaned[df_cleaned['Value'].isna()]

# Display the result
nan_rows

,ANZSCO code,Occupation,Year,Metric,Value
21,1392,Senior Non-Commissioned Defence Force Officers,2011,Employment,NaN
219,4411,Defence Force Members - Other Ranks,2011,Employment,NaN
375,1392,Senior Non-Commissioned Defence Force Officers,2012,Employment,NaN
573,4411,Defence Force Members - Other Ranks,2012,Employment,NaN
729,1392,Senior Non-Commissioned Defence Force Officers,2013,Employment,NaN
...,...,...,...,...,...
10026,2541,Midwives,None,Age 15 - 19,NaN
10028,2543,Nurse Managers,None,Age 15 - 19,NaN
10037,2711,Barristers,None,Age 15 - 19,NaN
10039,2713,Solicitors,None,Age 15 - 19,NaN


In [12]:
# Fill NaN values in the 'Value' column with 0
df_cleaned['Value'] = df_cleaned['Value'].fillna(0)

/var/folders/j5/j2851kd10mq66kby8zkqyyf40000gn/T/ipykernel_14492/2091216732.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Value'] = df_cleaned['Value'].fillna(0)


In [13]:
df_cleaned.to_csv('data/employment_outlook.csv',index=False)

# Earnings & Hours & Education

In [135]:
# Load the cleaned data from the specified sheet
file_path = 'data/occupation-profiles-data.xlsx'
data = pd.read_excel(file_path, sheet_name='Earnings_and_edu_clean')

In [136]:
data

,ANZSCO code,Occupation,industry,Industry_share,Median_FT_Weekly_Earnings,Average_FT_Hours_Worked,Post Grad/ Grad Dip or Grad Cert,Bachelor degree,Advanced Diploma/Diploma,Certificate III/IV,Year 12,Year 11,Year 10 and below
0,1111,Chief Executives and Managing Directors,"Professional, Scientific and Technical Services",17.1,NaN,52,22.8,31.1,12.3,13.7,12.3,2.5,5.3
1,1112,General Managers,Manufacturing,13.5,NaN,49,21.7,29.6,15.6,12.4,13.5,2.5,4.7
2,1113,Legislators,Public Administration and Safety,89.5,NaN,56,22.5,33.6,12.5,9.5,11.0,2.6,8.4
3,1211,Aquaculture Farmers,"Agriculture, Forestry and Fishing",76.5,NaN,49,3.3,15.7,9.2,24.0,16.2,6.2,25.5
4,1212,Crop Farmers,"Agriculture, Forestry and Fishing",95.1,NaN,52,2.4,9.9,9.2,18.5,18.7,8.0,33.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,899918,Sign Erectors,Construction,47.1,NaN,44,0.7,5.0,5.7,41.2,18.9,7.5,20.9
1232,899921,Ticket Collectors and Ushers,Arts and Recreation Services,40.9,NaN,43,3.0,13.8,8.2,10.7,36.5,9.6,18.2
1233,899922,Trolley Collectors,Retail Trade,56.6,NaN,42,2.9,7.6,6.0,9.5,38.7,9.5,25.9
1234,899923,Road Traffic Controllers,Public Administration and Safety,58.1,NaN,45,0.9,4.4,6.7,27.1,20.4,8.1,32.5


In [138]:
df_cleaned.to_csv('data/earnings_and_edu.csv',index=False)